In [28]:
import os
import dspy
import json

from typing import List, Dict

In [2]:
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openai_api_key = os.getenv('OPENAI_API_KEY')

In [17]:
config_gpt5 = '''
lm = dspy.LM(
        "openai/gpt-5-mini",
        model_type="responses",
        temperature=1.0,
        max_tokens=16000,
    )

dspy.settings.configure(lm=lm)
'''

lm = dspy.LM('openai/gpt-4o-mini', api_key=openai_api_key)
dspy.configure(lm=lm)

In [4]:
prompt_poesia = """
## FUNÇÃO
Você é um assistente de escrita criativa com foco em poesia.

## TAREFA
Sua tarefa é gerar associações de palavras que gerem novas ideias para a pessoa que está escrevendo.

Ao receber uma palavra, responda com uma série de palavras e expressões relacionadas a ela. Organize essas palavras em categorias ou temas. Inclua quantas outras achar necessário, da forma que achar melhor.
"""

In [18]:
resposta = lm(messages=[
    {"role": "system", "content": prompt_poesia},
    {"role": "user", "content": "Água"}]
)

In [19]:
resposta_formatada = "\n\n".join(resposta)
print(resposta_formatada)

### Elementos Naturais
- Rio
- Oceano
- Chuva
- Lago
- Fonte
- Gota
- Nuvem

### Emoções e Sensações
- Pureza
- Tranquilidade
- Fluidez
- Refresco
- Renovação
- Serenidade

### Ciclos da Vida
- Crescimento
- Fertilidade
- Ciclo da água
- Vida
- Sustento
- Transformação

### Metáforas e Simbolismos
- Espelho da alma
- Correnteza do tempo
- Fluxo das emoções
- Profundidade
- Reflexão
- Limpeza

### Atividades e Usos
- Natação
- Banho
- Hidratante
- Irrigação
- Navegação
- Pesca

### Cultura e Mitos
- Deuses da água
- Lendas de sereias
- Rituais de purificação
- Contos de inundação
- Mitos de criação

### Palavras Relacionadas
- Hidratação
- Transparência
- Movimento
- Ondas
- Maré
- Salinidade

Essas associações podem inspirar diferentes temas e imagens em sua poesia sobre a água!


In [20]:
lm.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': '\n## FUNÇÃO\nVocê é um assistente de escrita criativa com foco em poesia.\n\n## TAREFA\nSua tarefa é gerar associações de palavras que gerem novas ideias para a pessoa que está escrevendo.\n\nAo receber uma palavra, responda com uma série de palavras e expressões relacionadas a ela. Organize essas palavras em categorias ou temas. Inclua quantas outras achar necessário, da forma que achar melhor.\n'},
   {'role': 'user', 'content': 'Água'}],
  'kwargs': {},
  'response': ModelResponse(id='chatcmpl-CIinpGPkfuVxduSXHCi7vt7ysnJex', created=1758578161, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_560af6e559', choices=[Choices(finish_reason='stop', index=0, message=Message(content='### Elementos Naturais\n- Rio\n- Oceano\n- Chuva\n- Lago\n- Fonte\n- Gota\n- Nuvem\n\n### Emoções e Sensações\n- Pureza\n- Tranquilidade\n- Fluidez\n- Refresco\n- Renovação\n- Serenidade\n\n### Ciclos da Vida\n-

In [39]:
# 1) put the task description in the class docstring (this becomes the signature.instructions)
class TermosRelacionados(dspy.Signature):
    """
    Dada uma palavra de input, responda com uma série de palavras e expressões relacionadas a ela.
    Organize essas palavras em categorias ou temas. Inclua quantas outras achar necessário, da forma que achar melhor.
    Retorne apenas o objeto JSON, sem comentários, markdown etc.
    """
    palavra_original: str = dspy.InputField(desc="Uma única palavra, ex: 'Árvore'")
    termos_relacionados: dict[str, list[str]] = dspy.OutputField(desc="Categorias de termos relacionados e os termos relacionados em si")

# 2) (recommended) tell DSPy to use the JSONAdapter so the LM is asked for structured JSON
dspy.configure(adapter=dspy.JSONAdapter())

# 3) create the Predict module (no instructions kw here!)
generate = dspy.Predict(TermosRelacionados)

# 4) call it
pred = generate(palavra_original="Árvore")

# 5) convert Prediction -> dict and print as JSON
result_dict = pred.toDict()   # Example/Prediction utilities include toDict()
print(json.dumps(result_dict, indent=2, ensure_ascii=False))


{
  "termos_relacionados": {
    "Tipos de Árvores": [
      "Coníferas",
      "Decíduas",
      "Frutíferas",
      "Nativas",
      "Exóticas"
    ],
    "Partes da Árvore": [
      "Folhas",
      "Caule",
      "Raízes",
      "Copa",
      "Casca"
    ],
    "Usos da Árvore": [
      "Madeira",
      "Sombra",
      "Frutos",
      "Ornamentação",
      "Produção de oxigênio"
    ],
    "Ambientes": [
      "Florestas",
      "Parques",
      "Jardins",
      "Áreas urbanas",
      "Sítios"
    ],
    "Simbolismo": [
      "Vida",
      "Crescimento",
      "Força",
      "Sabedoria",
      "Conexão com a natureza"
    ]
  }
}
